In [1]:
!conda install  gensim  --yes
!conda install  tensorflow --yes
!conda install  keras --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/chainer_p36

  added / updated specs: 
    - gensim


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    gensim-3.4.0               |   py36h14c3975_0        21.5 MB
    smart_open-1.8.4           |             py_0          55 KB
    certifi-2019.6.16          |           py36_1         156 KB
    bz2file-0.98               |           py36_1          13 KB
    ------------------------------------------------------------
                                           Total:        21.7 MB

The following NEW packages will be INSTALLED:

    bz2file:         0.98-py36_1         
    gensim:          3.4.0-py36h14c397

In [5]:
import numpy as np
np.random.seed(13)

import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

import gensim

In [6]:
path = get_file('alice.txt', origin='http://www.gutenberg.org/files/11/11-0.txt')
#path = "/Users/cmauceri/PRG/notebooks/data/alice.txt"
corpus = open(path).readlines()[:300]
corpus = [sentence for sentence in corpus if sentence.count(' ') >= 2]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)
V = len(tokenizer.word_index) + 1
dim = 100
window_size = 2

180224/173595 [===============================] - 0s 0us/step


In [8]:
def generate_data(corpus, window_size, V):
    maxlen = window_size*2
    for words in corpus:
        L = len(words)
        for index, word in enumerate(words):
            contexts = []
            labels   = []            
            s = index - window_size
            e = index + window_size + 1
            
            contexts.append([words[i] for i in range(s, e) if 0 <= i < L and i != index])
            labels.append(word)

            x = sequence.pad_sequences(contexts, maxlen=maxlen)
            y = np_utils.to_categorical(labels, V)
            yield (x, y)

In [9]:
cbow = Sequential()
cbow.add(Embedding(input_dim=V, output_dim=dim, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
cbow.add(Dense(V, activation='softmax'))

W0814 19:53:18.092072 140310236575552 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/chainer_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0814 19:53:18.108888 140310236575552 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/chainer_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0814 19:53:18.111220 140310236575552 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/chainer_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [10]:
cbow.compile(loss='categorical_crossentropy', optimizer='adadelta')

W0814 19:53:23.610255 140310236575552 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/chainer_p36/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0814 19:53:23.629375 140310236575552 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/chainer_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [11]:
for ite in range(10):
    loss = 0.
    for x, y in generate_data(corpus, window_size, V):
        loss += cbow.train_on_batch(x, y)

    print(ite, loss)

W0814 19:53:27.046362 140310236575552 deprecation.py:323] From /home/ec2-user/anaconda3/envs/chainer_p36/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0814 19:53:27.145032 140310236575552 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/chainer_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



0 17412.568029403687
1 16152.25845336914
2 16024.157576084137
3 15933.220362067223
4 15825.345357060432
5 15722.70236581564
6 15632.665354698896
7 15551.543939858675
8 15473.690562710166
9 15396.474030569196


In [12]:
f = open('vectors.txt' ,'w')
f.write('{} {}\n'.format(V-1, dim))

8

In [13]:
vectors = cbow.get_weights()[0]
for word, i in tokenizer.word_index.items():
    str_vec = ' '.join(map(str, list(vectors[i, :])))
    f.write('{} {}\n'.format(word, str_vec))
f.close()

In [18]:
#!yes | pip3 uninstall ibm-cos-sdk-core 
#!pip3 install 'urllib3<1.25,>=1.20' --force-reinstall --upgrade  --no-cache-dir
#!python3 -m pip install --user boto3 --upgrade  --no-cache-dir
#!pip3 install awscli --force-reinstall --upgrade  --no-cache-dir
#!print("*")*30
#!pip3 install ibm-cos-sdk-core --upgrade   --no-cache-dir
print("*"* 100) 


****************************************************************************************************


In [14]:
import boto3
import gensim
w2v = gensim.models.KeyedVectors.load_word2vec_format('./vectors.txt', binary=False)

/home/ec2-user/anaconda3/envs/chainer_p36/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [15]:
w2v.most_similar(positive=['rabbit'])

[('under', 0.7948824167251587),
 ('through', 0.7905367612838745),
 ('into', 0.777804970741272),
 ('hole', 0.7497458457946777),
 ('passage', 0.748123288154602),
 ('same', 0.7418513298034668),
 ('words', 0.7247788906097412),
 ('right', 0.7152592539787292),
 ('little', 0.7066874504089355),
 ('of', 0.7059344053268433)]

In [16]:
w2v.most_similar(positive=['alice'])

[('she', 0.6100988388061523),
 ('you', 0.6004399061203003),
 ('poor', 0.5680834054946899),
 ('them', 0.5612019300460815),
 ('that', 0.5537208318710327),
 ('eat', 0.5532217025756836),
 ('now', 0.5522719025611877),
 ('dark', 0.5486494302749634),
 ('marked', 0.5483137369155884),
 ('thought', 0.5456308126449585)]